Decision Tree Classifier Building in Scikit-learn


Importing Required Libraries


In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn import metrics 

Loading Data

In [3]:
titanic = pd.read_csv(r"C:\Users\purva\Downloads\The Titanic dataset.csv")

In [3]:
titanic.head()

,1,2,3,4,5,6,7,8,9,10
0,sn,pclass,survived,NaN,gender,age,family,fare,embarked,date
1,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
2,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
3,2,3,0,Master. Eugene Joseph,male,?,2,20.25,S,02-Jan-90
4,3,2,0,"Abbott, Mr. Rossmore Edward",NaN,NaN,2,**,S,03-Jan-90


In [4]:
titanic.describe()

,1,2,3,4,5,6,7,8,9,10
count,1302,1302,1302,1301,1301,1045,1300,1300,1296,1302
unique,1301,4,3,1297,3,98,10,283,4,1301
top,1,3,0,"Connolly, Miss. Kate",male,24,0,13,S,01-Jan-90
freq,2,700,801,2,835,47,781,59,905,2


Feature Selection

In [5]:
titanic.columns = ['sn', 'Pclass', 'Survived', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

feature_cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = titanic[feature_cols]  
y = titanic['Survived']    


In [6]:
titanic = titanic.drop(index=titanic.index[0])


In [7]:
titanic.head()

,sn,Pclass,Survived,Name,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
2,1,3,0,Mr. Anthony,male,42,0,7.55,NaN,01-Jan-90
3,2,3,0,Master. Eugene Joseph,male,?,2,20.25,S,02-Jan-90
4,3,2,0,"Abbott, Mr. Rossmore Edward",NaN,NaN,2,**,S,03-Jan-90
5,4,3,1,"Abbott, Mr. Rossmore Edward",female,35,2,20.25,S,04-Jan-90


Splitting Data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test


Building Decision Tree Model

In [10]:
X = pd.get_dummies(titanic[feature_cols], drop_first=True)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(X.mean())

y = y.loc[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


Evaluating the Model

In [11]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.7749360613810742


In [13]:
print(len(feature_cols))
print(clf.n_features_in_)  
print(clf.classes_)        

7
1689
['0' '1']


In [14]:
from sklearn.tree import export_graphviz
from io import StringIO  

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, 
                feature_names=X.columns[:clf.n_features_in_],  
                class_names=[str(cls) for cls in clf.classes_])    


In [15]:
from sklearn.tree import DecisionTreeClassifier
print(type(clf))


<class 'sklearn.tree._classes.DecisionTreeClassifier'>


In [16]:
print(dot_data.getvalue())


digraph Tree {
node [shape=box, style="filled, rounded", color="black", fontname="helvetica"] ;
edge [fontname="helvetica"] ;
0 [label=<Sex_male &le; 0.5<br/>gini = 0.464<br/>samples = 910<br/>value = [577, 333]<br/>class = 0>, fillcolor="#f4caab"] ;
1 [label=<Pclass_3 &le; 0.5<br/>gini = 0.404<br/>samples = 321<br/>value = [90.0, 231.0]<br/>class = 1>, fillcolor="#86c3ef"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label=<Embarked_06-Mar-93 &le; 0.5<br/>gini = 0.116<br/>samples = 162<br/>value = [10, 152]<br/>class = 1>, fillcolor="#46a3e7"] ;
1 -> 2 ;
3 [label=<Embarked_30-Jun-91 &le; 0.5<br/>gini = 0.106<br/>samples = 161<br/>value = [9, 152]<br/>class = 1>, fillcolor="#45a3e7"] ;
2 -> 3 ;
4 [label=<Embarked_13-Jan-92 &le; 0.5<br/>gini = 0.095<br/>samples = 160<br/>value = [8, 152]<br/>class = 1>, fillcolor="#43a2e6"] ;
3 -> 4 ;
5 [label=<Embarked_04-Mar-91 &le; 0.5<br/>gini = 0.084<br/>samples = 159<br/>value = [7.0, 152.0]<br/>class = 1>, fillcolor="#42a2e6

Visualizing Decision Trees

In [17]:
from sklearn.tree import export_graphviz
from io import StringIO
from IPython.display import Image
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, 
                feature_names=X.columns[:clf.n_features_in_],   
                class_names=[str(cls) for cls in clf.classes_]) 


graph = pydotplus.graph_from_dot_data(dot_data.getvalue())


if graph:
    graph.write_png('titanic_tree.png')
    display(Image('titanic_tree.png'))
else:
    print("Graph creation failed!")


In [18]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.7672634271099744
